# The Task

## Project Overview

In the file dataset/data.csv, you will find a dataset containing news articles with the following columns:

label: 0 if the news is fake, 1 if the news is real.
title: The headline of the news article.
text: The full content of the article.
subject: The category or topic of the news.
date: The publication date of the article.
Your goal is to build a classifier that is able to distinguish between the two.

Once you have a classifier built, then use it to predict the labels for dataset/validation_data.csv. Generate a new file where the label 2 has been replaced by 0 (fake) or 1 (real) according to your model. Please respect the original file format, do not include extra columns, and respect the column separator.

Please ensure to split the data.csv into training and test datasets before using it for model training or evaluation.

Guidance
Like in a real life scenario, you are able to make your own choices and text treatment. Use the techniques you have learned and the common packages to process this data and classify the text.

Deliverables
Python Code: Provide well-documented Python code that conducts the analysis.
Predictions: A csv file in the same format as validation_data.csv but with the predicted labels (0 or 1)
Accuracy estimation: Provide the teacher with your estimation of how your model will perform.
Presentation: You will present your model in a 10-minute presentation. Your teacher will provide further instructions.

# Import

In [ ]:
import re
import nltk
import string
import numpy as np
import pandas as pd
import seaborn as sns
from nltk import pos_tag
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras import regularizers
from sklearn.metrics import RocCurveDisplay
from tensorflow.keras.optimizers import Adam
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.models import Sequential
from nltk.stem.snowball import SnowballStemmer
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.backend import clear_session
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer, WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, auc, roc_curve

# Loading the Data

In [ ]:
data = pd.read_csv('../dataset/training_data_lowercase.csv', sep='\t', names=['labels', 'text'])

In [ ]:
data

# Undertanding the Data

In [ ]:
data.info()

In [ ]:
print('Its unique values are ',data['labels'].unique())
print(print(data['labels'].describe()))

In [ ]:
plt.hist(data.labels, color='red')
plt.show()

In [ ]:
data.head()

In [ ]:
data.tail()

# Preprocessing the Data

## Basic cleaning 

In [ ]:
def clean_html_text(text: str) -> str:
    if text is None:
        return ''
    text = str(text)
    # Remove inline JavaScript/CSS
    text = re.sub(r"(?is)<script.*?>.*?</script>", " ", text)
    text = re.sub(r"(?is)<style.*?>.*?</style>", " ", text)
    # Remove HTML comments
    text = re.sub(r"(?s)<!--.*?-->", " ", text)
    # Remove the remaining tag
    text = re.sub(r"(?s)<[^>]+>", " ", text)
    # Remove prefixed b
    text = re.sub(r"^\s*b[\"'](.+?)[\"']\s*$", r"\1", text)
    # Remove video
    # text = re.sub(r"\s*\[video\]$", r"\1", text)    
    # Remove end of the line characters
    text = re.sub(r"\s*[\[\(][^\]\)]+[\]\)]\s*$", "", text)    
    # Remove \t from middle and end of the texts
    text = re.sub(r"\b\\t"," ",text)
    # Remove \t from startof the texts
    text = re.sub(r"^\\t"," ",text)
    # Remove all the special characters and numbers
    text = re.sub(r"[^A-Za-z\s]", " ", text)
    # Remove all single characters
    text = re.sub(r"\b[A-Za-z]\b", " ", text)
    # Remove single characters from the start
    text = re.sub(r"^[A-Za-z]\s+", " ", text)
    # Substitute multiple spaces with single space
    text = re.sub(r"\s+", " ", text).strip()
    # Convert to lowercase
    text = text.lower()
    return text

punct_pattern = f"[{re.escape(string.punctuation)}]"

In [ ]:
data['pre_text'] = data['text'].astype(str).apply(lambda x: clean_html_text(x))
data['pre_text'] = data['pre_text'].astype(str).apply(lambda x: re.sub(punct_pattern, "", x))
data['pre_text'] = data['pre_text'].astype(str).apply(lambda x: word_tokenize(x))
data.head()

## Removing stop words

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
data['pre_text'] = data['pre_text'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

In [ ]:
bag_of_words = {}

for lista in data['pre_text']:
    for word in lista:
        if bag_of_words == 0:
            bag_of_words[word] = 1
        elif word in bag_of_words:
            bag_of_words[word] +=1
        else:
            bag_of_words[word] = 1

print(sorted(bag_of_words.items(), key=lambda x: -x[1])[:100])

In [ ]:
words_to_filter = ['video','says', 'tweets', 'tells','screenshots',
                   'details', 'fck', 'btch', 'images', 'cck', 'image'
                   ,'videos','ahole']

In [ ]:
data['pre_text_filter'] = data['pre_text'].apply(lambda tokens: [word for word in tokens if word not in words_to_filter])

In [ ]:
bag_of_words = {}

for lista in data['pre_text_filter']:
    for word in lista:
        if bag_of_words == 0:
            bag_of_words[word] = 1
        elif word in bag_of_words:
            bag_of_words[word] +=1
        else:
            bag_of_words[word] = 1

print(sorted(bag_of_words.items(), key=lambda x: -x[1])[:100])

### Using Stemmer

#### Snowball

In [ ]:
snowball = SnowballStemmer('english')

In [ ]:
data['snow_text'] = data['pre_text'].apply(lambda tokens: [snowball.stem(token) for token in tokens])

#### Porter

In [ ]:
porter = PorterStemmer()

In [ ]:
data['porter_text'] = data['pre_text'].apply(lambda tokens: [porter.stem(token) for token in tokens])

### Using Lemmatizer

In [ ]:
lemm = WordNetLemmatizer()

In [ ]:
data['lemm_text'] = data['pre_text'].apply(lambda tokens: [lemm.lemmatize(token) for token in tokens])

In [ ]:
data.info()

# Spliting the data into Training and Test

In [ ]:
X = data.iloc[:,2:]

In [ ]:
y = data.iloc[:,0]

In [ ]:
print(X.shape, y.shape)

## Using only the preprocessed text

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X['pre_text'], y, test_size=0.2, random_state=42)

## Using the preprocessed text + snow stemmer

In [ ]:
X_train_snow, X_test_snow, y_train_snow, y_test_snow = train_test_split(X['snow_text'], y, test_size=0.2, random_state=42)

## Using the preprocessed text + porter stemmer

In [ ]:
X_train_porter, X_test_porter, y_train_porter, y_test_porter = train_test_split(X['porter_text'], y, test_size=0.2, random_state=42)

## Using the preprocessed text + noise removal

In [ ]:
X_train_filt, X_test_filt, y_train_filt, y_test_filt = train_test_split(X['pre_text_filter'], y, test_size=0.2, random_state=42)

## Using the preprocessed text + lemmatizer

In [ ]:
X_train_lemm, X_test_lemm, y_train_lemm, y_test_lemm = train_test_split(X['lemm_text'], y, test_size=0.2, random_state=42)

# Defining a plotting function for Evaluation phase

In [ ]:
def plot_learning_curve(
    model,
    X,
    y,
    scoring="accuracy",
    cv=5,
    train_sizes=np.linspace(0.1, 1.0, 5),
    title=None
):
    train_sizes, train_scores, val_scores = learning_curve(
        model,
        X,
        y,
        scoring=scoring,
        cv=cv,
        train_sizes=train_sizes,
        n_jobs=-1
    )

    train_mean = train_scores.mean(axis=1)
    val_mean = val_scores.mean(axis=1)

    plt.figure()
    plt.plot(train_sizes, train_mean, marker="o", label="Training score")
    plt.plot(train_sizes, val_mean, marker="o", label="Validation score")
    plt.xlabel("Training set size")
    plt.ylabel(scoring)
    plt.title(title or model.__class__.__name__)
    plt.legend()
    plt.grid(True)
    plt.show()

# Training some classifiers
##### Best( Only preprocessed text - BOW Logistic Regression - Acc: 93.96 %, Gini: 96.31 %)

### Only preprocessed text - Best(BOW Logistic Regression - Acc: 93.96 %, Gini: 96.31 %)

#### TF-IDF - Best (Logistic Regression - Acc: 92.7 %, Gini: 96.12 %)

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_tfidf, y_train)
y_hat = dt_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test, y_hat))

y_proba = dt_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("DecisionTreeClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = dt_classifier,
    X = X_tfidf,
    y = y_train,
    scoring="f1",
    title="DecisionTree Classifier - TFIDF (Preprocessed Text)"
)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_tfidf, y_train)
y_hat = log_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test, y_hat))

y_proba = log_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("LogisticRegression")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = log_classifier,
    X = X_tfidf,
    y = y_train,
    scoring="f1",
    title="Logistic Regression Classifier - TF-IDF (Preprocessed Text)"
)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_tfidf, y_train)
y_hat = nb_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test, y_hat))

y_proba = nb_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("MultinomialNB")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = nb_classifier,
    X = X_tfidf,
    y = y_train,
    scoring="f1",
    title="Naive Bayes Classifier - TF-IDF (Preprocessed Text)"
)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_tfidf, y_train)
y_hat = rf_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test, y_hat))

y_proba = rf_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("RandomForestClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = rf_classifier,
    X = X_bow,
    y = y_train,
    scoring="f1",
    title="Random Forest Classifier - TF-IDF (Preprocessed Text)"
)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=50, random_state=42, n_jobs=-1).fit(X_tfidf, y_train)
y_hat = pac.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test, y_hat))

y_proba = pac.decision_function(X_test_tfidf)
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

plot_learning_curve(
    model = pac,
    X = X_tfidf,
    y = y_train,
    scoring="f1",
    title="PassiveAggressive Classifier - TF-IDF (Preprocessed Text)"
)

#### BoW - Best (Naive Bayes - Acc: 93.01 %, Gini: 96.31 % )

In [ ]:
vectorizer = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_bow, y_train)
y_hat = dt_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test, y_hat))

y_proba = dt_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("DecisionTreeClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = dt_classifier,
    X = X_bow,
    y = y_train,
    scoring="f1",
    title="DecisionTree Classifier - Bow (Preprocessed Text)"
)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_bow, y_train)
y_hat = log_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test, y_hat))

y_proba = log_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("LogisticRegression")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = log_classifier,
    X = X_bow,
    y = y_train,
    scoring="f1",
    title="Logistic Regression Classifier - Bow (Preprocessed Text)"
)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_bow, y_train)
y_hat = nb_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))

y_proba = nb_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("MultinomialNB")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = nb_classifier,
    X = X_tfidf,
    y = y_train,
    scoring="f1",
    title="Naive Bayes Classifier - Bow (Preprocessed Text)"
)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_bow, y_train)
y_hat = rf_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test, y_hat))

y_proba = rf_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("RandomForestClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = rf_classifier,
    X = X_bow,
    y = y_train,
    scoring="f1",
    title="Random Forest Classifier - Bow (Preprocessed Text)"
)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=50, random_state=42, n_jobs=-1).fit(X_bow, y_train)
y_hat = pac.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test, y_hat))

y_proba = pac.decision_function(X_test_bow)
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

plot_learning_curve(
    model = pac,
    X = X_bow,
    y = y_train,
    scoring="f1",
    title="Passive Aggressive Classifier - Bow (Preprocessed Text)"
)

### Preprocessed text + noise removal - Best(Bow Logistic Regression - Acc: 92.8 %, Gini: 96.04 %)

#### TF-IDF - Best (Naive Bayes - Acc: 92.57 %, Gini: 95.89 % )¶

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_tfidf = vectorizer.fit_transform(X_train_filt)
X_test_tfidf = vectorizer.transform(X_test_filt)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_tfidf, y_train_filt)
y_hat = dt_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = dt_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_tfidf, y_train_filt)
y_hat = log_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = log_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_tfidf, y_train_filt)
y_hat = nb_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = nb_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_filt, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("MultinomialNB")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_filt, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = nb_classifier,
    X = X_tfidf,
    y = y_train,
    scoring="f1",
    title="Naive Bayes Classifier - TF-IDF (Preprocessed Text filter)"
)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_tfidf, y_train_filt)
y_hat = rf_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = rf_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=50, random_state=42, n_jobs=-1).fit(X_tfidf, y_train_filt)
y_hat = pac.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = pac.decision_function(X_test_tfidf)
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_filt, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_filt, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### BoW - Best (Logistic Regression - Acc: 92.8 %, Gini: 96.04 % )¶

In [ ]:
vectorizer = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_bow = vectorizer.fit_transform(X_train_filt)
X_test_bow = vectorizer.transform(X_test_filt)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_bow, y_train_filt)
y_hat = dt_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = dt_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_bow, y_train_filt)
y_hat = log_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = log_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_filt, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("LogisticRegression")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_filt, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = log_classifier,
    X = X_bow,
    y = y_train,
    scoring="f1",
    title="Logistic Regression Classifier - Bow (Preprocessed Text)"
)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_bow, y_train_filt)
y_hat = nb_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = nb_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_bow, y_train_filt)
y_hat = rf_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = rf_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=50, random_state=42, n_jobs=-1).fit(X_bow, y_train_filt)
y_hat = pac.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_filt, y_hat))
print("Classification Report:\n", classification_report(y_test_filt, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_filt, y_hat))

y_proba = pac.decision_function(X_test_bow)
fpr, tpr, thresholds = roc_curve(y_test_filt, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)


plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_filt, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_filt, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Preprocessed text + snow stemmer - Best(Bow Logistic Regression - Acc: 92.51 %, Gini: 96.04 %)

#### TF-IDF - Best (Naive Bayes - Acc: 91.94 %, Gini: 95.39 % )¶

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_tfidf = vectorizer.fit_transform(X_train_snow)
X_test_tfidf = vectorizer.transform(X_test_snow)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_tfidf, y_train_snow)
y_hat = dt_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = dt_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_tfidf, y_train_snow)
y_hat = log_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = log_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_tfidf, y_train_snow)
y_hat = nb_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = nb_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_snow, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("MultinomialNB")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_snow, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = nb_classifier,
    X = X_tfidf,
    y = y_train,
    scoring="f1",
    title="Naive Bayes Classifier - TF-IDF (Preprocessed Text Snow Lemmatizer)"
)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_tfidf, y_train_snow)
y_hat = rf_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = rf_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=50, random_state=42, n_jobs=-1).fit(X_tfidf, y_train_snow)
y_hat = pac.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = pac.decision_function(X_test_tfidf)
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_snow, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_snow, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### BoW - Best (Logistic Regression - Acc: 92.51 %, Gini: 96.04 % )

In [ ]:
vectorizer = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_bow = vectorizer.fit_transform(X_train_snow)
X_test_bow = vectorizer.transform(X_test_snow)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_bow, y_train_snow)
y_hat = dt_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = dt_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_bow, y_train_snow)
y_hat = log_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = log_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_snow, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("LogisticRegression")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_snow, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = log_classifier,
    X = X_bow,
    y = y_train,
    scoring="f1",
    title="Logistic Regression Classifier - Bow (Preprocessed Text + snow)"
)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_bow, y_train_snow)
y_hat = nb_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = nb_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_bow, y_train_snow)
y_hat = rf_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = nb_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=100, random_state=42, n_jobs=-1).fit(X_bow, y_train_snow)
y_hat = pac.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_snow, y_hat))
print("Classification Report:\n", classification_report(y_test_snow, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_snow, y_hat))

y_proba = pac.decision_function(X_test_bow)
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_snow, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_snow, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Preprocessed text + porter stemmer - Best(Bow Logistic Regression - Acc: 92.56 %, Gini: 96.02 %)

#### TF-IDF - Best (Passive Agressive Classifier - Acc: 91.97 %, Gini: 95.45 % )¶

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_tfidf = vectorizer.fit_transform(X_train_porter)
X_test_tfidf = vectorizer.transform(X_test_porter)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_tfidf, y_train_porter)
y_hat = dt_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = dt_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_porter, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_tfidf, y_train_porter)
y_hat = log_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = log_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_porter, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_tfidf, y_train_porter)
y_hat = nb_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = nb_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_porter, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_porter, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("MultinomialNB")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_porter, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = nb_classifier,
    X = X_tfidf,
    y = y_train,
    scoring="f1",
    title="Naive Bayes Classifier - TF-IDF (Preprocessed Text porter)"
)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_tfidf, y_train_porter)
y_hat = rf_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = rf_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_porter, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=50, random_state=42, n_jobs=-1).fit(X_tfidf, y_train_porter)
y_hat = pac.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = pac.decision_function(X_test_tfidf)
fpr, tpr, thresholds = roc_curve(y_test_porter, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_porter, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_porter, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### BoW - Best (Logistic Regression - Acc: 92.56 %, Gini: 96.02 % )¶

In [ ]:
vectorizer = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_bow = vectorizer.fit_transform(X_train_porter)
X_test_bow = vectorizer.transform(X_test_porter)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_bow, y_train_porter)
y_hat = dt_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = dt_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_bow, y_train_porter)
y_hat = log_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = log_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_porter, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_porter, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("LogisticRegression")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_porter, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = log_classifier,
    X = X_bow,
    y = y_train,
    scoring="f1",
    title="Logistic Regression Classifier - Bow (Preprocessed Text + porter)"
)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_bow, y_train_porter)
y_hat = nb_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = nb_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_bow, y_train_porter)
y_hat = rf_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = rf_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_snow, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=100, random_state=42, n_jobs=-1).fit(X_bow, y_train_porter)
y_hat = pac.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_porter, y_hat))
print("Classification Report:\n", classification_report(y_test_porter, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_porter, y_hat))

y_proba = pac.decision_function(X_test_bow)
fpr, tpr, thresholds = roc_curve(y_test_porter, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_porter, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_porter, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Preprocessed text + lemmatizer - Best (Bow Naive Bayes - Acc: 92.76 %, Gini: 96.15 %

#### TF-IDF - Best (Naive Bayes - Acc: 92.47 %, Gini: 95.90 % )¶

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_tfidf = vectorizer.fit_transform(X_train_lemm)
X_test_tfidf = vectorizer.transform(X_test_lemm)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_tfidf, y_train_lemm)
y_hat = dt_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = dt_classifier.predict_proba(X_test_tfidf)[:,1]
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_tfidf, y_train_lemm)
y_hat = log_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = log_classifier.predict_proba(X_test_tfidf)[:,1]
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_tfidf, y_train_lemm)
y_hat = nb_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = nb_classifier.predict_proba(X_test_tfidf)[:,1]
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_lemm, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("MultinomialNB")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_lemm, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = nb_classifier,
    X = X_tfidf,
    y = y_train,
    scoring="f1",
    title="Naive Bayes Classifier - TF-IDF (Preprocessed Text lemmatizer)"
)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_tfidf, y_train_lemm)
y_hat = rf_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = rf_classifier.predict_proba(X_test_tfidf)[:,1]
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=50, random_state=42, n_jobs=-1).fit(X_tfidf, y_train_lemm)
y_hat = pac.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = pac.decision_function(X_test_tfidf)
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_lemm, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_lemm, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### BoW - Best (Logistic Regression - Acc: 92.76 %, Gini: 96.15 % )¶

In [ ]:
vectorizer = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False
)

X_bow = vectorizer.fit_transform(X_train_lemm)
X_test_bow = vectorizer.transform(X_test_lemm)

##### Decision Tree metrics

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42).fit(X_bow, y_train_lemm)
y_hat = dt_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = dt_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Logistic Regression metrics

In [ ]:
log_classifier = LogisticRegression(random_state=42).fit(X_bow, y_train_lemm)
y_hat = log_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = log_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_lemm, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("LogisticRegression")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_lemm, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


plot_learning_curve(
    model = log_classifier,
    X = X_bow,
    y = y_train,
    scoring="f1",
    title="Logistic Regression Classifier - Bow (Preprocessed Text + porter)"
)

##### Naive Bayes metrics

In [ ]:
nb_classifier = MultinomialNB().fit(X_bow, y_train_lemm)
y_hat = nb_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = nb_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### Random Forest metrics

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1).fit(X_bow, y_train_lemm)
y_hat = rf_classifier.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = rf_classifier.predict_proba(X_test_bow)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

##### PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=50, random_state=42, n_jobs=-1).fit(X_bow, y_train_lemm)
y_hat = pac.predict(X_test_bow)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test_lemm, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = pac.decision_function(X_test_bow)
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

plt.figure(figsize=(5, 4))
sns.heatmap(
    confusion_matrix(y_test_lemm, y_hat),
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("PassiveAggressiveClassifier")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_lemm, y_proba,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# N-grams 

##### TF-IDF - lemm - Logistic Regression - Acc: 92.65 %, Gini: 95.82 % )¶

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    lowercase=False,
    # create word unigrams + bigrams
    ngram_range=(1, 2),
    # drops rare tokens/bigrams that appear once
    min_df=2,
    # drops near-constant boilerplate tokens
    max_df=0.9
)

X_tfidf = vectorizer.fit_transform(X_train_porter)
X_test_tfidf = vectorizer.transform(X_test_porter)

In [ ]:
log_reg = LogisticRegression(random_state=42).fit(X_tfidf, y_train_lemm)
y_hat = log_reg.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test_lemm, y_hat))
print("Classification Report:\n", classification_report(y_test, y_hat))
print('Confusion matrix:\n', confusion_matrix(y_test_lemm, y_hat))

y_proba = log_reg.decision_function(X_test_tfidf)
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_proba)
print()
print('Gini coef.:', 2*(auc(fpr, tpr))-1)

In [ ]:
plot_learning_curve(
    model = log_reg,
    X = X_tfidf,
    y = y_train_lemm,
    scoring="f1",
    title="Logistic Regression - TF-IDF (Unigrams + Bigrams)"
)

# Embeddings

##### TF-IDF - lemm - Logistic Regression - Acc: 93.67 %, Gini: 96.96 % )¶

## Preparing the train test data

In [ ]:
clear_session()

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

# Create and train the Tokenizer
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train_lemm) # Learn from training text

# Convert text to numbers
X_train_seq = tokenizer.texts_to_sequences(X_train_lemm)
X_test_seq = tokenizer.texts_to_sequences(X_test_lemm)

# Selecting the MAX_LEN from the 90 % percentile
lengths = [len(seq) for seq in X_train_seq]
MAX_LEN = int(np.percentile(lengths, 90))

# Padding -all sequences same length-
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

## Creating the model

In [ ]:
# - MODEL DEFINITION

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=MAX_LEN),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.2),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dense(1, activation='sigmoid')
])

# Adding early stop based on val loss
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

# Reducing the step if we reach a plateau
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=2, min_lr=1e-4
)


model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss='binary_crossentropy', # for True/False
    metrics=['accuracy']
)

## Training the model

In [ ]:
history = model.fit(
    X_train_pad, y_train_lemm,
    epochs=50,
    batch_size=128,
    callbacks=[early_stop, reduce_lr],
    validation_split=.2,
    verbose=1)

model.summary()

## Metrics

In [ ]:
test_loss, test_acc = model.evaluate(X_test_pad, y_test_lemm , verbose=1)
print("Test accuracy:", test_acc)
print("Test loss:", test_loss)

plt.figure(figsize=(5, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Binary Cross-Entropy')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(5, 4))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Binary Cross-Entropy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Predict
y_pred_keras = model.predict(X_test_pad).ravel()

# Gini
fpr, tpr, thresholds = roc_curve(y_test_lemm, y_pred_keras)
roc_auc = auc(fpr, tpr)
gini = 2 * roc_auc - 1

print(f"ROC AUC: {roc_auc:.4f}")
print(f"Gini Coefficient: {gini:.4f}")

acc = history.history['val_accuracy'][-1]
loss = history.history['val_loss'][-1]

print("Validation Accuracy:", acc)
print("Validation Loss:", loss)
print("Test Accuracy:", test_acc)
print("Test Loss:", test_loss)

In [ ]:
print("Classification Report:\n", classification_report(y_test_lemm, y_pred_labels))
y_pred_labels = (y_pred_keras >= 0.5).astype(int)
cm = confusion_matrix(y_test_lemm, y_pred_labels)

plt.figure(figsize=(5, 4))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Embeddings model")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))

RocCurveDisplay.from_predictions(
    y_test_lemm,
    y_pred_keras,
    name=f"Model (AUC = {roc_auc:.3f})",
    ax=ax
)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# sd